# Training a NN with Urban Sound Challenge Data

**Background:**

In parallel to the actuall fall detection experiment, we'll train a neural network leveraging the urban challenge dataset in order to develop the necessary libraries to map wav files to features (either MFCC or spectogram) and baseline the performance of the different approaches)

## Libraries and File Locations

In [1]:
import matplotlib.pyplot as plt
from python_speech_features import mfcc
import pandas as pd
import os
import numpy as np
import soundfile
from scipy import signal

In [2]:
URBAN_SOUND_DIR = "/media/nvidia/ROMULO'S/urban_sound_challenge/"

## Load sample WAV File

Test that features can be generated in a couple of ways

* MFCC
* Spectrograms

**MFCCs**

In [3]:
samples, sample_rate = soundfile.read(URBAN_SOUND_DIR+"Train/5132.wav")

In [4]:
mfcc_feat = mfcc(samples,sample_rate,nfft=2400)

In [5]:
mfcc_feat.shape

(68, 13)

**Spectrogram**

In [6]:
f, t, spectrogram= signal.spectrogram(samples,sample_rate)

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/scipy/signal/spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 2, using nperseg = 2
  .format(nperseg, input_length))


In [8]:
spectrogram.shape

(16573, 2, 1)

## Training using MFCC Loading the dataset

In [9]:
train_df = pd.read_csv(URBAN_SOUND_DIR+'train.csv')
test_df = pd.read_csv(URBAN_SOUND_DIR+'test.csv')

In [10]:
def parse_mfcc(row,folder,nfft):
    # fun
    #ction to load files and extract features
    file_name = os.path.join(URBAN_SOUND_DIR, folder, str(row.ID) + '.wav')
   # handle exception to check if there isn't a file which is corrupted
    try:
        samples, sample_rate = soundfile.read(file_name)
        # here kaiser_fast is a technique used for faster extraction
        mfcc_feat = mfcc(samples,sample_rate,nfft=nfft)
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return pd.Series([None, None])
    return pd.Series([mfcc_feat, row.Class])

In [13]:
temp = train_df.apply(parse_mfcc,args=('Train',2400), axis=1)
temp.columns = ['feature', 'label']

In [14]:
temp.dropna(inplace=True)

In [15]:
assert temp.shape == (5435,2)

## Define the classes and encode the training set

In [16]:
classes = list(temp['label'].unique())

In [17]:
temp['label_encoded'] = temp['label'].apply(classes.index)

### Save the dataframe for future models/training sessions

In [20]:
temp.to_csv(URBAN_SOUND_DIR+"train_mfcc_transformed.csv")

## Aggregate MFCC series into a single dimension

In [18]:
X_avg = []
for x in temp.feature.tolist():
    X_avg.append(np.mean(x,axis=0))

In [19]:
X_avg[0].shape

(13,)

In [21]:
X = np.array(X_avg)
y = np.array(temp.label_encoded.tolist())

In [22]:
set([x.shape for x in X]) == set([(13,)])

True

# Setting up a simple model

In [23]:
import tensorflow as tf
from tensorflow import keras

In [24]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(13,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(60, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [25]:
model.compile(optimizer='sgd', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.fit(X, y, epochs=10,batch_size=10,shuffle=False)

Epoch 1/10
5435/5435 [==============================] - 16s 3ms/step - loss: 1.4945 - acc: 0.5345
Epoch 2/10
5435/5435 [==============================] - 13s 2ms/step - loss: 0.9592 - acc: 0.6942
Epoch 3/10
5435/5435 [==============================] - 13s 2ms/step - loss: 0.7842 - acc: 0.7512
Epoch 4/10
5435/5435 [==============================] - 13s 2ms/step - loss: 0.6809 - acc: 0.7849
Epoch 5/10
5435/5435 [==============================] - 13s 2ms/step - loss: 0.6085 - acc: 0.8098: 3s - lo
Epoch 6/10
5435/5435 [==============================] - 13s 2ms/step - loss: 0.5518 - acc: 0.8243
Epoch 7/10
5435/5435 [==============================] - 13s 2ms/step - loss: 0.5047 - acc: 0.8403
Epoch 8/10
5435/5435 [==============================] - 13s 2ms/step - loss: 0.4627 - acc: 0.8557
Epoch 9/10
5435/5435 [==============================] - 13s 2ms/step - loss: 0.4252 - acc: 0.8651
Epoch 10/10
5435/5435 [==============================] - 14s 3ms/step - loss: 0.3923 - acc: 0.8771: 0s - loss

In [27]:
## Save the model to JSON

In [28]:
# serialize model to JSON
model_json = model.to_json()
with open("./trained_models/hdf5/urban_sound_model.json", "w") as json_file:
    json_file.write(model_json)

In [29]:
# serialize weights to HDF5
model.save_weights("./trained_models/hdf5/urban_sound_model.h5")
print("Saved model to disk")

ImportError: `save_weights` requires h5py when saving in hdf5.

# Test the MFCC model

In [ ]:
test_temp = test_df.apply(parser_test,args=(('Test',4800)), axis=1)
test_temp.columns = ['feature']

# Simple model using spectrograms

In [1]:
def parser_spec(row):
    # fun
    #ction to load files and extract features
    folder = 'Train'
    file_name = os.path.join(URBAN_SOUND_DIR, folder, str(row.ID) + '.wav')
   # handle exception to check if there isn't a file which is corrupted
    try:
        samples, sample_rate = soundfile.read(file_name)
        # here kaiser_fast is a technique used for faster extraction
        f, t, spectrogram= signal.spectrogram(samples,sample_rate)
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return pd.Series([None, None])
    return pd.Series([spectrogram, row.Class])

In [2]:
temp_spec = train_df.apply(parser_spec, axis=1)
temp_spec.columns = ['feature', 'label']

NameError: name 'train_df' is not defined